
# 处理丢失数据

有两种丢失数据：
- None
- np.nan(NaN)

In [3]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

In [ ]:
None np.nan

## 1. None

None是Python自带的，其类型为python object。因此，None不能参与到任何计算中。

In [1]:
type(None)

NoneType

In [2]:
None + 1

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [4]:
n = np.array([1,2,3,None])
n

array([1, 2, 3, None], dtype=object)

In [5]:
n.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

object类型的运算要比int类型的运算慢得多  
计算不同数据类型求和时间  
%timeit np.arange(1e5,dtype=xxx).sum()

In [6]:
%timeit np.arange(1e5, dtype='int').sum()

282 µs ± 79 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
%timeit np.arange(1e5, dtype='float').sum()

356 µs ± 57.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
%timeit np.arange(1e5, dtype='object').sum()

13.9 ms ± 1.9 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 2. np.nan（NaN）

np.nan是浮点类型，能参与到计算中。但计算的结果总是NaN。

但可以使用np.nan*()函数来计算nan，此时视nan为0。

In [9]:
type(np.nan)

float

In [10]:
1 + np.nan

nan

In [11]:
n = np.array([1,2,3,4, np.nan])
n

array([ 1.,  2.,  3.,  4., nan])

In [12]:
np.nansum(n)

10.0

## 3. pandas中的None与NaN

### 1) pandas中None与np.nan都视作np.nan

创建DataFrame

In [34]:
index = ['张三', '李四', '王五', '赵六']
columns = ['语文', '数学', '英语', 'python']
data = np.random.randint(0, 150, size=(4,4))
df = DataFrame(index=index, columns=columns, data=data)
df

,语文,数学,英语,python
张三,23,14,47,15
李四,70,85,100,108
王五,12,142,113,84
赵六,0,56,6,38


In [35]:
df.loc['张三', '数学'] = None

In [15]:
df

,语文,数学,英语,python
张三,147,NaN,8,122
李四,5,134.0,48,6
王五,25,71.0,33,53
赵六,58,123.0,138,111


In [36]:
df.loc['李四', '语文'] = np.nan
df

,语文,数学,英语,python
张三,23.0,NaN,47,15
李四,NaN,85.0,100,108
王五,12.0,142.0,113,84
赵六,0.0,56.0,6,38


使用DataFrame行索引与列索引修改DataFrame数据

### 2) pandas中None与np.nan的操作

- ``isnull()``
- ``notnull()``
- ``dropna()``: 过滤丢失数据
- ``fillna()``: 填充丢失数据

In [ ]:
1, 先检测一下哪些数据是NaN. isnull(), notnull
2, 根据情况进行处理,删除NaN数据.dropna
3, 填充或者替换数据, fillna

(1)判断函数
- ``isnull()``
- ``notnull()``

In [17]:
df

,语文,数学,英语,python
张三,147.0,NaN,8,122
李四,NaN,134.0,48,6
王五,25.0,71.0,33,53
赵六,58.0,123.0,138,111


In [18]:
df.isnull()

,语文,数学,英语,python
张三,False,True,False,False
李四,True,False,False,False
王五,False,False,False,False
赵六,False,False,False,False


In [19]:
s = Series([1,2,3, None])
s

0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

In [20]:
s.isnull()

0    False
1    False
2    False
3     True
dtype: bool

In [21]:
df.notnull()

,语文,数学,英语,python
张三,True,False,True,True
李四,False,True,True,True
王五,True,True,True,True
赵六,True,True,True,True


In [22]:
s.notnull()

0     True
1     True
2     True
3    False
dtype: bool

In [25]:
df

,语文,数学,英语,python
张三,147.0,NaN,8,122
李四,NaN,134.0,48,6
王五,25.0,71.0,33,53
赵六,58.0,123.0,138,111


In [ ]:
df.any()

In [24]:
# 结合any和all, 判断行列是否有空数据.
# 判断列是否有空数据
df.isnull().any(axis=0)

语文         True
数学         True
英语        False
python    False
dtype: bool

In [26]:
df.isnull().any(axis=1)

张三     True
李四     True
王五    False
赵六    False
dtype: bool

(2) 过滤函数
- ``dropna()``

In [29]:
df.loc['张三', '语文'] = None
df

,语文,数学,英语,python
张三,NaN,NaN,8,122
李四,NaN,134.0,48,6
王五,25.0,71.0,33,53
赵六,58.0,123.0,138,111


In [30]:
# axis控制删除行还是列, axis=0表示删除行,
# how控制怎么删, any表示只要有空删除. all全是空才删除. 
#subset  表示子集  过滤的范围
df.dropna(how='all', subset=['语文', '数学'])

,语文,数学,英语,python
李四,NaN,134.0,48,6
王五,25.0,71.0,33,53
赵六,58.0,123.0,138,111


In [32]:
df.dropna(axis=1, inplace=True)

In [33]:
df

,英语,python
张三,8,122
李四,48,6
王五,33,53
赵六,138,111


可以选择过滤的是行还是列（默认为行）

也可以选择过滤的方式 how = 'all'

(3) 填充函数 Series/DataFrame
- ``fillna()``

In [37]:
df

,语文,数学,英语,python
张三,23.0,NaN,47,15
李四,NaN,85.0,100,108
王五,12.0,142.0,113,84
赵六,0.0,56.0,6,38


In [38]:
# fillna有两个用法.
# 第一个用法: 用指定的值来填充.
df.fillna(value=100)

,语文,数学,英语,python
张三,23.0,100.0,47,15
李四,100.0,85.0,100,108
王五,12.0,142.0,113,84
赵六,0.0,56.0,6,38


In [41]:
# 第二个用法, 用已有的值填充.
# method : {'backfill', 'bfill', 'pad', 'ffill', None}
# backfill bfill 向后填充
# pad, ffill向前填充 forward
# 前后关系要根据axis来定
df.fillna(axis=1, method='ffill')

,语文,数学,英语,python
张三,23.0,23.0,47.0,15.0
李四,NaN,85.0,100.0,108.0
王五,12.0,142.0,113.0,84.0
赵六,0.0,56.0,6.0,38.0


In [42]:
df.loc['李四', '数学'] = np.nan
df

,语文,数学,英语,python
张三,23.0,NaN,47,15
李四,NaN,NaN,100,108
王五,12.0,142.0,113,84
赵六,0.0,56.0,6,38


In [45]:
df.fillna(axis=0, method='bfill', limit=1)

,语文,数学,英语,python
张三,23.0,NaN,47,15
李四,12.0,142.0,100,108
王五,12.0,142.0,113,84
赵六,0.0,56.0,6,38


In [46]:
df.loc['王五', '数学'] = np.nan
df.loc['李四', '数学'] = 100
df

,语文,数学,英语,python
张三,23.0,NaN,47,15
李四,NaN,100.0,100,108
王五,12.0,NaN,113,84
赵六,0.0,56.0,6,38


In [47]:
# limit 限制连续填充的次数
df.fillna(axis=0, method='bfill', limit=2)

,语文,数学,英语,python
张三,23.0,100.0,47,15
李四,12.0,100.0,100,108
王五,12.0,56.0,113,84
赵六,0.0,56.0,6,38


可以选择前向填充还是后向填充

对于DataFrame来说，还要选择填充的轴axis。记住，对于DataFrame来说：

- axis=0：index/行
- axis=1：columns/列

============================================

练习7：

1. 简述None与NaN的区别

2. 假设张三李四参加模拟考试，但张三因为突然想明白人生放弃了英语考试，因此记为None，请据此创建一个DataFrame,命名为ddd3

3. 老师决定根据用数学的分数填充张三的英语成绩，如何实现？
    用李四的英语成绩填充张三的英语成绩？

============================================

In [ ]:
1, None是python自带的对象, 不能参与计算. NaN是浮点型,可以参与计算 ,但是结果总是NaN, 可以用np.nan*去消除np.nan的影响, 在pandas中None和np.nan都被当成np.nan处理..

In [48]:
df

,语文,数学,英语,python
张三,23.0,NaN,47,15
李四,NaN,100.0,100,108
王五,12.0,NaN,113,84
赵六,0.0,56.0,6,38


In [49]:
df.loc['张三', '英语'] = None
df

,语文,数学,英语,python
张三,23.0,NaN,NaN,15
李四,NaN,100.0,100.0,108
王五,12.0,NaN,113.0,84
赵六,0.0,56.0,6.0,38


In [50]:
df.fillna(axis=1, method='ffill')

,语文,数学,英语,python
张三,23.0,23.0,23.0,15.0
李四,NaN,100.0,100.0,108.0
王五,12.0,12.0,113.0,84.0
赵六,0.0,56.0,6.0,38.0


In [51]:
df.fillna(axis=0, method='bfill')

,语文,数学,英语,python
张三,23.0,100.0,100.0,15
李四,12.0,100.0,100.0,108
王五,12.0,56.0,113.0,84
赵六,0.0,56.0,6.0,38
